In [1]:
import pandas as pd

import time
import random
import re
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
pip install webdriver-manager


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install selenium



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False  # To ensure minus sign is displayed correctly
df = pd.read_csv('/Users/emanon/Downloads/vis/project/top200_movies.csv')
df.head()

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,국적,배급사,장르,러닝타임,평점,감독
0,1,명량,2014-07-30,"135,762,515,310","17,616,371","1,587","188,730",한국,한국,(주)씨제이이엔엠,액션,128분,8.88,김한민
1,2,극한직업,2019-01-23,"139,657,209,516","16,266,491","2,003","292,819",한국,한국,(주)씨제이이엔엠,코미디,111분,9.20,이병헌
2,3,신과함께-죄와 벌,2017-12-20,"115,727,528,087","14,414,658","1,912","214,631",한국,한국,롯데쇼핑㈜롯데엔터테인먼트,판타지,139분,8.73,김용화
3,4,국제시장,2014-12-17,"110,960,515,230","14,266,489","1,044","212,728",한국,한국,(주)씨제이이엔엠,드라마,126분,9.16,윤제균
4,5,어벤져스: 엔드게임,2019-04-24,"122,490,648,020","13,977,409","2,835","246,411",미국,미국,월트디즈니컴퍼니코리아 유한책임회사,액션,181분,9.50,안소니 루소


In [5]:
movie_names = df['영화명'].dropna().tolist()
movie_names

['명량',
 '극한직업',
 '신과함께-죄와 벌',
 '국제시장',
 '어벤져스: 엔드게임',
 '겨울왕국 2',
 '베테랑',
 '아바타',
 '서울의 봄',
 '도둑들',
 '7번방의 선물',
 '알라딘',
 '암살',
 '범죄도시2',
 '광해, 왕이 된 남자',
 '신과함께-인과 연',
 '택시운전사',
 '파묘',
 '부산행',
 '범죄도시4',
 '변호인',
 '해운대',
 '어벤져스: 인피니티 워',
 '괴물',
 '아바타: 물의 길',
 '범죄도시3',
 '왕의 남자',
 '어벤져스: 에이지 오브 울트론',
 '인터스텔라',
 '겨울왕국',
 '기생충',
 '보헤미안 랩소디',
 '검사외전',
 '엑시트',
 '설국열차',
 '관상',
 '아이언맨 3',
 '인사이드 아웃 2',
 '캡틴 아메리카: 시빌 워',
 '해적: 바다로 간 산적',
 '수상한 그녀',
 '백두산',
 '탑건: 매버릭',
 '과속스캔들',
 '국가대표',
 '스파이더맨: 파 프롬 홈',
 '디워',
 '공조',
 '트랜스포머 3',
 '히말라야',
 '스파이더맨: 노 웨이 홈',
 '베테랑2',
 '미션임파서블:고스트프로토콜',
 '밀정',
 '최종병기 활',
 '트랜스포머',
 '트랜스포머: 패자의 역습',
 '써니',
 '한산: 용의 출현',
 '스파이더맨: 홈 커밍',
 '엘리멘탈',
 '1987',
 '베를린',
 '마스터',
 '터널',
 '어벤져스',
 '내부자들',
 '인천상륙작전',
 '공조2: 인터내셔날',
 '럭키',
 '은밀하게 위대하게',
 '범죄도시',
 '곡성',
 '화려한 휴가',
 '좋은 놈, 나쁜 놈, 이상한 놈',
 '늑대소년',
 '군함도',
 '미션 임파서블: 폴아웃',
 '웰컴 투 동막골',
 '다크 나이트 라이즈',
 '사도',
 '아저씨',
 '킹스맨 : 시크릿 에이전트',
 '미션 임파서블: 로그네이션',
 '미녀는 괴로워',
 '전우치',
 '연평해전',
 '인셉션',
 '레미제라블',
 '닥터 스트레인지: 대혼돈의 멀티버스'

In [6]:
chrome_options = webdriver.ChromeOptions()
url = 'https://www.naver.com'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(2)
driver.get(url)

In [25]:
rate_list = []

for movie in movie_names:
    search_url = f"http://www.cine21.com/search/?q={movie}"
    driver.get(search_url)
    time.sleep(random.uniform(2, 3))

    try:
        # 전문가 평점 추출
        rate_element = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[3]/div[1]/ul[2]/li/div/div[1]/span")
        rate_html = rate_element.get_attribute("innerHTML")
        rate = rate_html.split("<span")[0].strip()
    except:
        rate = None

    rate_list.append(rate)

In [26]:
rate_list

['6.18',
 '6.40',
 '5.75',
 '5.33',
 '7.17',
 '6.25',
 '7.38',
 '5.67',
 '6.67',
 '7.00',
 '5.00',
 '7.24',
 '6.25',
 '6.60',
 '7.00',
 '6.25',
 '5.83',
 '7.15',
 '7.10',
 '4.60',
 '7.20',
 '6.00',
 '7.00',
 '8.00',
 '7.67',
 '5.50',
 '6.67',
 '6.88',
 '7.67',
 '7.00',
 '9.09',
 '6.17',
 '5.57',
 '7.17',
 '7.67',
 '6.00',
 '7.40',
 '7.25',
 '7.45',
 '5.75',
 '5.33',
 '5.75',
 '8.60',
 '6.71',
 '6.33',
 '6.50',
 '4.80',
 '4.63',
 '5.00',
 '5.43',
 '7.00',
 '6.80',
 '7.50',
 '7.44',
 '6.50',
 '5.25',
 '6.00',
 '6.25',
 '6.80',
 '7.00',
 '7.29',
 '8.11',
 '8.00',
 '9.00',
 '3.00',
 '7.33',
 '6.00',
 '3.00',
 '5.33',
 '5.00',
 '4.80',
 '6.00',
 '6.00',
 '6.25',
 '7.00',
 '6.00',
 '6.22',
 '7.43',
 '7.67',
 '7.00',
 '1.00',
 '6.25',
 '7.43',
 '7.38',
 '5.00',
 '6.13',
 '4.75',
 '7.75',
 '5.00',
 '6.67',
 '7.00',
 '7.17',
 '6.50',
 '4.86',
 '4.67',
 '5.71',
 '6.80',
 '7.60',
 '6.00',
 '6.75',
 '6.33',
 '6.83',
 '5.71',
 '6.29',
 '7.00',
 '6.75',
 '5.43',
 '7.00',
 '6.80',
 '4.14',
 '6.40',
 

In [7]:
rate_list2 = []

for movie in movie_names:
    search_url = f"http://www.cine21.com/search/?q={movie}"
    driver.get(search_url)
    time.sleep(random.uniform(2, 3))
    
    # 전문가 평점 추출
    try:
        expert_element = driver.find_element(
            By.XPATH, 
            "//div[@class='sub_info search_star']/div[@class='star_area_s'][1]/span[@class='num']"
        )
        expert_rating = expert_element.text
    except Exception as e:
        expert_rating = None
    
    # 네티즌 평점 추출
    try:
        netizen_element = driver.find_element(
            By.XPATH, 
            "//div[@class='sub_info search_star']/div[@class='star_area_s'][2]/span[@class='num']"
        )
        netizen_rating = netizen_element.text
    except Exception as e:
        netizen_rating = None

    # 두 평점을 딕셔너리로 저장
    ratings = {'movie': movie, 'expert': expert_rating, 'netizen': netizen_rating}
    rate_list2.append(ratings)

driver.quit()

# 결과 출력
for rate in rate_list2:
    print(rate)

{'movie': '명량', 'expert': '6.18', 'netizen': '7.20'}
{'movie': '극한직업', 'expert': '6.40', 'netizen': '7.19'}
{'movie': '신과함께-죄와 벌', 'expert': '5.75', 'netizen': '5.58'}
{'movie': '국제시장', 'expert': '5.33', 'netizen': '7.20'}
{'movie': '어벤져스: 엔드게임', 'expert': '7.17', 'netizen': '8.26'}
{'movie': '겨울왕국 2', 'expert': '6.25', 'netizen': '8.11'}
{'movie': '베테랑', 'expert': '7.38', 'netizen': '7.92'}
{'movie': '아바타', 'expert': '5.67', 'netizen': '8.76'}
{'movie': '서울의 봄', 'expert': '6.67', 'netizen': '8.36'}
{'movie': '도둑들', 'expert': '7.00', 'netizen': '7.20'}
{'movie': '7번방의 선물', 'expert': '5.00', 'netizen': '6.81'}
{'movie': '알라딘', 'expert': '7.24', 'netizen': '8.55'}
{'movie': '암살', 'expert': '6.25', 'netizen': '7.55'}
{'movie': '범죄도시2', 'expert': '6.60', 'netizen': '7.67'}
{'movie': '광해, 왕이 된 남자', 'expert': '7.00', 'netizen': '7.98'}
{'movie': '신과함께-인과 연', 'expert': '6.25', 'netizen': '6.40'}
{'movie': '택시운전사', 'expert': '5.83', 'netizen': '6.78'}
{'movie': '파묘', 'expert': '7.15', 'netizen

In [8]:
df_rate = pd.DataFrame(rate_list2)
df_rate.head()


,movie,expert,netizen
0,명량,6.18,7.20
1,극한직업,6.40,7.19
2,신과함께-죄와 벌,5.75,5.58
3,국제시장,5.33,7.20
4,어벤져스: 엔드게임,7.17,8.26


In [9]:
print("Number of rows in df_rate:", df_rate.shape[0])

Number of rows in df_rate: 200


In [10]:
df_rate.to_csv('df_rate.csv', index=False)